In [1]:
import requests
import json
import mercury as mr

app = mr.App(title="Image tags evals", static_notebook=True)

mercury.App

In [13]:
url = "http://localhost:9200/_cluster/settings"
headers = {
    'Content-Type': 'application/json'
}

payload = {
    "persistent": {
        "plugins": {
            "ml_commons": {
                "only_run_on_ml_node": "false",
                "model_access_control_enabled": "true",
                "native_memory_threshold": "99"
            }
        }
    }
}


response = requests.request("PUT", url, headers=headers, data=json.dumps(payload))

In [14]:
mr.JSON(response.json(), level=4)

In [15]:
url = "http://localhost:9200/_plugins/_ml/model_groups/_register"

payload = {
  "name": "TEXT_model_group",
  "description": "A model group for text embeddings."
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)

model_group_id = response.json()['model_group_id']

print(f"Created Model Group ID {model_group_id}")

Created Model Group ID zH6GYZQBrLHGb1uD4Qll


In [16]:
url = "http://localhost:9200/_plugins/_ml/models/_register"

payload = {
  "name": "huggingface/sentence-transformers/all-MiniLM-L6-v2",
  "version": "1.0.1",
  "model_group_id": model_group_id,
  "model_format": "TORCH_SCRIPT"
}


response = requests.post(url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)
task_id = response.json()['task_id']

In [17]:
url = f"http://localhost:9200/_plugins/_ml/tasks/{task_id}"

max_attempts = 5
attempts = 0

state = None
while state != 'COMPLETED' and attempts < max_attempts:
    time.sleep(5) # wait five second and then check again.  We are downloading msmarco model from huggingface.
    response = requests.request("GET", url, headers=headers)
    #mr.JSON(response.json(), level=4)
    print(response.json()['state'])
    state = response.json()['state']
    attempts += 1

model_id = response.json()['model_id']

print(f"Created Model ID {model_id}")

COMPLETED
Created Model ID zn6HYZQBrLHGb1uDUQni


In [18]:
url = f"http://localhost:9200/_plugins/_ml/models/{model_id}/_deploy"

response = requests.post(url, headers=headers)
mr.JSON(response.json(), level=4)
deploy_model_task_id = response.json()['task_id']

In [19]:
url = f"http://localhost:9200/_plugins/_ml/tasks/{deploy_model_task_id}"

max_attempts = 5
attempts = 0

state = None
while state != 'COMPLETED' and attempts < max_attempts:
    attempts += 1
    time.sleep(5) # wait five second and then check again.  Deploying the model can take a while
    response = requests.request("GET", url, headers=headers)
    #mr.JSON(response.json(), level=4)
    print(response.json()['state'])
    state = response.json()['state']
    attempts += 1    

model_id = response.json()['model_id']

print(f"Finished Deploying Model ID {model_id}")

COMPLETED
Finished Deploying Model ID zn6HYZQBrLHGb1uDUQni


In [20]:
url = f"http://localhost:9200/_plugins/_ml/_predict/text_embedding/{model_id}"

payload = {
  "text_docs":["This is a test string."],
  "return_number": True,
  "target_response": ["sentence_embedding"]
}


response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=5)

In [21]:
url = "http://localhost:9200/_ingest/pipeline/tags-ingest-pipeline"

payload = {
  "description": "A text embedding pipeline",
  "processors": [
    {
      "text_embedding": {
        "model_id": model_id,
        "field_map": {
          "tags": "tag_embedding" # input field --> vector field
        }
      }
    }
  ]
}


response = requests.put(url, headers=headers, data=json.dumps(payload))
mr.JSON(response.json(), level=4)